In [3]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET

# Set the paths for the image and annotation folders
image_folder = 'oxford-iiit-pet/images'
annotation_folder = 'oxford-iiit-pet/annotations/xmls'
output_folder = 'segmentation_maps'


In [4]:
import os
import cv2
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision import models

def decode_segmap(image, source, nc=3):
    label_colors = np.array([(0, 0, 0), (255, 0, 0), (0, 255, 0), (0, 0, 255)])
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)

    for l in range(0, nc):
        idx = image == l
        r[idx] = label_colors[l, 0]
        g[idx] = label_colors[l, 1]
        b[idx] = label_colors[l, 2]

    rgb = np.stack([r, g, b], axis=2)
    return rgb

def segment(net, path):
    img = Image.open(path)
    trf = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    inp = trf(img).unsqueeze(0)
    out = net(inp)['out']
    om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
    return om

dlab = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()

# Set the paths for the image and output folders
image_folder = 'images'
output_folder = 'segmentation_maps'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate over image files in the image folder
for image_file in os.listdir(image_folder):
    # Check if the file is an image file
    if image_file.endswith(('.png', '.jpg', '.jpeg')):
        # Segment the image
        image_path = os.path.join(image_folder, image_file)
        segmap = segment(dlab, image_path)

        # Create a tricolor segmentation map
        segmap_color = decode_segmap(segmap)

        # Save the segmentation map
        mask_file = os.path.splitext(image_file)[0] + '_segmap.png'
        cv2.imwrite(os.path.join(output_folder, mask_file), cv2.cvtColor(segmap_color, cv2.COLOR_RGB2BGR))


Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
